In [123]:
class PDF(object):
    def __init__(self, pdf, size=(1200,600)):
        self.pdf = pdf
        self.size = size

    def _repr_html_(self):
        return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

    def _repr_latex_(self):
        return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)


In [124]:
PDF('data/Ecoli/bw_glucose_mass_reclaimed_genome.pdf')

In [125]:
PDF('data/Ecoli/bw_glucose_mass_reclaimed_metabolism.pdf')

# Max Mass Reclamation

`MaxMassReclamation` is the `MinGenome` algorithm with the following modified objective function:



$$\max\sum_{k\in K}y_k \sum_{i=1}^k{m}_i - \sum_{k\in K} x_k\sum_{i=1}^k{m}_i$$

where $m_i$ is the measured protein mass of gene $i$ and $\sum_{i=1}^k{m}_i$ is the cumulative protein mass of all genes from the origin of replication to the $k$th gene.  Since $\sum_{k\in K}x_k = 1$ and  By subtracting the cumulative protein mass of the start gene from the cumulative protein mass of the end gene, we obtain the cumulative protein mass of the interval for the optimal solution

We maximize the mass that is knocked out based on absolute quantitative proteomics [Schmidt et al 2015](http://www.nature.com/articles/nbt.3418) in $(fg/cell)$


We then run `Max mass reclamation` iteratively without replacement.


Instead of maximizing the length of the knockout,

$$\max\sum_{k\in K}y_k {d'}_k -\sum_{k\in K} x_k d_k$$

where the variables are

$$x_k = \begin{array}{l}
1, \text{ if gene or promoter $k$ is the first gene or promoter within the deleted segment  }\\
0, \text{ otherwise}
\end{array}
$$
and
$$y_k = \begin{array}{l}
1, \text{ if gene or promoter $k$ is immediately after the end of the deleted segment  }\\
0, \text{ otherwise}
\end{array}
$$
where 

$$\sum_{k\in K}x_k = 1$$
and $$\sum_{k\in K}y_k = 1$$


and the parameters are

* $d_k$: Position of the first nucleotide of the delted sequence starting from the origin of replication when gen eor pmoter $k$ is selected to be deleted in the begining of the stretch.  Note that $d_k$ is not always the start site of a gene or promoter. It is the first nucleotide of the nonoverlapped region between the gene/promoter $k$ and gene/promoter $k-1$
* ${d'}_k$: Position of the first nucleotide of the gene or promoter
k immediately after the deleted sequence

![Definitions of the start site and end site of a deletion in MinGenome algorithm](Figure8.jpg)




In [22]:
import pandas as pd
df = pd.read_excel('data/Ecoli/genes_and_promoters.xlsx', sheet_name='all_clear_v2')
df

start, end = 190, 9191
genes_of_interval = df[(df['start'] >= start) & \
                       (df['end'] <= end) & \
                       (df['class'] == 'gene')]
genes_of_interval['gene_or_promoter'].tolist()

[u'b0001',
 u'b0002',
 u'b0003',
 u'b0004',
 u'b0005',
 u'b0006',
 u'b0007',
 u'b0008']

In [23]:
df

,gene_or_promoter,start,end,strand,class,genes_in_TU,start_if_select_as_start,cannot_as_start
0,PM00249,148,189,1,promoter,"[b0001, b0002, b0003, b0004]",148,0
1,b0001,190,255,1,gene,NaN,190,0
2,b0002,337,2799,1,gene,NaN,337,0
3,b0003,2801,3733,1,gene,NaN,2801,0
4,b0004,3734,5020,1,gene,NaN,3734,0
5,b0005,5234,5530,1,gene,NaN,5234,0
6,b0006,5683,6459,-1,gene,NaN,5683,0
7,b0007,6529,7959,-1,gene,NaN,6529,0
8,PM0-9956,8191,8237,1,promoter,[b0008],8191,0
9,b0008,8238,9191,1,gene,NaN,8238,0


In [97]:
proteins_fg = pd.read_excel('data/Ecoli/Schmidt/nbt.3418-S2.xlsx', sheet_name='Table S13', skiprows=3)
proteins_fg

,Uniprot Accession,Gene,Description,Cellular protein location (according to www.uniprot.org),A14.07032,A14.07033,A14.07034,A14.07036,A14.07037,A14.07038,...,A14.07115,A14.07117,A14.07118,A14.07119,A14.07121,A14.07122,A14.07123,A14.07125,A14.07126,A14.07127
0,P04825,pepN,Aminopeptidase N OS=Escherichia coli (strain K...,Cell inner membrane,4.633803e-01,4.113094e-01,3.915993e-01,2.136611e-01,2.127355e-01,2.102780e-01,...,2.619060e-01,1.605115e-01,1.762474e-01,1.532321e-01,1.972634e-01,2.219234e-01,2.172675e-01,2.251897e-01,2.415721e-01,2.495236e-01
1,P0C0V0,degP,Protease do OS=Escherichia coli (strain K12) G...,Cell inner membrane,2.128826e-01,2.076923e-01,2.117216e-01,2.587472e-01,2.753042e-01,2.732864e-01,...,1.983017e-01,1.340973e-01,1.583725e-01,1.395959e-01,1.359015e-01,1.784184e-01,1.805699e-01,3.353371e-01,3.702367e-01,3.175771e-01
2,P0AAI3,ftsH,ATP-dependent zinc metalloprotease FtsH OS=Esc...,Cell inner membrane,4.640302e-01,4.143873e-01,4.943970e-01,2.440695e-01,2.623177e-01,2.724735e-01,...,2.041780e-01,1.721386e-01,1.648425e-01,1.837950e-01,1.980768e-01,2.045779e-01,2.061072e-01,2.451916e-01,2.672380e-01,2.739067e-01
3,P0ABC7,hflK,Modulator of FtsH protease HflK OS=Escherichia...,Cell inner membrane,1.772662e-01,1.711126e-01,1.828374e-01,9.544338e-02,1.122174e-01,1.019477e-01,...,9.121105e-02,6.832720e-02,5.526516e-02,6.789463e-02,7.846795e-02,8.284558e-02,8.471687e-02,1.138036e-01,1.236418e-01,1.141445e-01
4,P08506,dacC,D-alanyl-D-alanine carboxypeptidase dacC OS=Es...,Cell inner membrane,8.301700e-02,5.860804e-02,6.915716e-02,6.989639e-02,7.144231e-02,7.433892e-02,...,1.000501e-01,9.289993e-02,9.475696e-02,9.951884e-02,6.972466e-02,9.001456e-02,9.822825e-02,1.029425e-01,1.144356e-01,1.114165e-01
5,P0ABC3,hflC,Modulator of FtsH protease HflC OS=Escherichia...,Cell inner membrane,1.300773e-01,1.328812e-01,1.410357e-01,7.555463e-02,8.089087e-02,8.058817e-02,...,6.409134e-02,5.087432e-02,5.364527e-02,5.651761e-02,6.374615e-02,6.534776e-02,7.848423e-02,8.499128e-02,8.734255e-02,9.080262e-02
6,P0AEB2,dacA,D-alanyl-D-alanine carboxypeptidase dacA OS=Es...,Cell inner membrane,1.552737e-01,1.336722e-01,1.445040e-01,1.008689e-01,9.944921e-02,9.930472e-02,...,6.377710e-02,4.638581e-02,5.268501e-02,4.432679e-02,7.098773e-02,6.368619e-02,6.319260e-02,8.758517e-02,9.556234e-02,8.581793e-02
7,P0AG14,sohB,Probable protease sohB OS=Escherichia coli (st...,Cell inner membrane,2.783188e-02,2.469125e-02,3.184970e-02,2.659054e-02,2.699793e-02,2.712781e-02,...,2.533363e-02,1.572704e-02,1.355775e-02,1.584925e-02,1.831426e-02,2.265361e-02,2.310298e-02,3.180126e-02,4.520411e-02,4.042833e-02
8,P23865,prc,Tail-specific protease OS=Escherichia coli (st...,Cell inner membrane,7.892680e-02,7.982559e-02,7.756057e-02,6.429820e-02,6.502070e-02,6.326422e-02,...,5.010172e-02,4.358141e-02,4.393447e-02,4.231139e-02,5.313490e-02,5.614253e-02,5.380110e-02,6.429808e-02,6.086548e-02,7.163990e-02
9,P23894,htpX,Protease HtpX OS=Escherichia coli (strain K12)...,Cell inner membrane,2.854932e-02,3.011286e-02,2.742700e-02,2.133862e-02,2.654830e-02,2.387221e-02,...,1.062292e-02,9.383213e-03,1.036593e-02,1.237462e-02,1.836186e-02,1.611691e-02,1.531539e-02,2.066662e-02,2.192009e-02,3.154345e-02


In [106]:
BW25113_fg = proteins_fg[['Uniprot Accession', 
                          'Description', 'Gene', 
                          'Cellular protein location (according to www.uniprot.org)', 
                          'A14.07036', 'A14.07037', 'A14.07038' ]].set_index('Uniprot Accession')
BW25113_fg

,Description,Gene,Cellular protein location (according to www.uniprot.org),A14.07036,A14.07037,A14.07038
Uniprot Accession,,,,,,
P04825,Aminopeptidase N OS=Escherichia coli (strain K...,pepN,Cell inner membrane,2.136611e-01,2.127355e-01,2.102780e-01
P0C0V0,Protease do OS=Escherichia coli (strain K12) G...,degP,Cell inner membrane,2.587472e-01,2.753042e-01,2.732864e-01
P0AAI3,ATP-dependent zinc metalloprotease FtsH OS=Esc...,ftsH,Cell inner membrane,2.440695e-01,2.623177e-01,2.724735e-01
P0ABC7,Modulator of FtsH protease HflK OS=Escherichia...,hflK,Cell inner membrane,9.544338e-02,1.122174e-01,1.019477e-01
P08506,D-alanyl-D-alanine carboxypeptidase dacC OS=Es...,dacC,Cell inner membrane,6.989639e-02,7.144231e-02,7.433892e-02
P0ABC3,Modulator of FtsH protease HflC OS=Escherichia...,hflC,Cell inner membrane,7.555463e-02,8.089087e-02,8.058817e-02
P0AEB2,D-alanyl-D-alanine carboxypeptidase dacA OS=Es...,dacA,Cell inner membrane,1.008689e-01,9.944921e-02,9.930472e-02
P0AG14,Probable protease sohB OS=Escherichia coli (st...,sohB,Cell inner membrane,2.659054e-02,2.699793e-02,2.712781e-02
P23865,Tail-specific protease OS=Escherichia coli (st...,prc,Cell inner membrane,6.429820e-02,6.502070e-02,6.326422e-02


In [112]:
import numpy as np
bw_glucose = ['A14.07036', 'A14.07037', 'A14.07038']
b2u = pd.read_table('data/Ecoli/blatter-to-uniprot.tab', index_col = 'Blattner')
ecoli = df.join(b2u, on='gene_or_promoter').join(BW25113_fg, on='Uniprot')


medianAbundance = ecoli[bw_glucose[0]].dropna().median()
display(medianAbundance)
ecoli.loc[ecoli['class']== 'promoter', bw_glucose[0]] = 0
ecoli[bw_glucose[0]].fillna(medianAbundance,inplace=True)
ecoli['cumulativeMass'] = np.cumsum(ecoli[bw_glucose[0]])
                          
ecoli.to_csv('data/Ecoli/{}_cumulative_mass.tab'.format(bw_glucose[0]), sep='\t', index=False)
ecoli

0.017911945107084437

,gene_or_promoter,start,end,strand,class,genes_in_TU,start_if_select_as_start,cannot_as_start,Uniprot,Description,Gene,Cellular protein location (according to www.uniprot.org),A14.07036,A14.07037,A14.07038,cumulativeMass
0,PM00249,148,189,1,promoter,"[b0001, b0002, b0003, b0004]",148,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.000000
1,b0001,190,255,1,gene,NaN,190,0,P0AD86,NaN,NaN,NaN,0.017912,NaN,NaN,0.017912
2,b0002,337,2799,1,gene,NaN,337,0,P00561,NaN,NaN,NaN,0.017912,NaN,NaN,0.035824
3,b0003,2801,3733,1,gene,NaN,2801,0,P00547,Homoserine kinase OS=Escherichia coli (strain ...,thrB,Cytoplasm,0.033131,0.032556,0.033302,0.068954
4,b0004,3734,5020,1,gene,NaN,3734,0,P00934,NaN,NaN,NaN,0.017912,NaN,NaN,0.086866
5,b0005,5234,5530,1,gene,NaN,5234,0,P75616,NaN,NaN,NaN,0.017912,NaN,NaN,0.104778
6,b0006,5683,6459,-1,gene,NaN,5683,0,P0A8I3,NaN,NaN,NaN,0.017912,NaN,NaN,0.122690
7,b0007,6529,7959,-1,gene,NaN,6529,0,P30143,NaN,NaN,NaN,0.017912,NaN,NaN,0.140602
8,PM0-9956,8191,8237,1,promoter,[b0008],8191,0,NaN,NaN,NaN,NaN,0.000000,NaN,NaN,0.140602
9,b0008,8238,9191,1,gene,NaN,8238,0,P0A870,Transaldolase B OS=Escherichia coli (strain K1...,talB,Cytoplasm,0.620909,0.622423,0.610794,0.761511


In [ ]:
proteins = pd.read_excel('data/Ecoli/Schmidt/nbt.3418-S2.xlsx', sheet_name='Table S9', skiprows=2)
mg1655 = proteins[['Uniprot Accession', 'Description', 'Gene', 'proteinMW', 'Copies/Cell_MG1655.Glucose' ]]


In [27]:
mg1655 = proteins[['Uniprot Accession', 
                   'Description', 
                   'Gene', 
                   'proteinMW', 
                   'Copies/Cell_MG1655.Glucose' ]].\
                iloc[:2019].\
                set_index('Uniprot Accession')
mg1655['g/Protein'] = mg1655['proteinMW'] * mg1655['Copies/Cell_MG1655.Glucose']
mg1655.sort_values('g/Protein', ascending=False)

,Description,Gene,proteinMW,Copies/Cell_MG1655.Glucose,g/Protein
Uniprot Accession,,,,,
P0CE47,Elongation factor Tu 1 OS=Escherichia coli (st...,tufA,43238.30279,279772.591884,1.209689e+10
P0A910,Outer membrane protein A OS=Escherichia coli (...,ompA,37159.64679,108747.026756,4.041001e+09
P25665,5-methyltetrahydropteroyltriglutamate--homocys...,metE,84602.80905,37939.368305,3.209777e+09
P0A6M8,Elongation factor G OS=Escherichia coli (strai...,fusA,77514.47523,40158.649597,3.112877e+09
P0ABK5,Cysteine synthase A OS=Escherichia coli (strai...,cysK,34450.28634,65474.668058,2.255621e+09
P05793,Ketol-acid reductoisomerase OS=Escherichia col...,ilvC,54016.26705,34730.063498,1.875988e+09
P04949,Flagellin OS=Escherichia coli (strain K12) GN=...,fliC,51246.56095,34899.631529,1.788486e+09
P08200,Isocitrate dehydrogenase [NADP] OS=Escherichia...,icd,45709.46787,37682.155126,1.722431e+09
P0A6P9,Enolase OS=Escherichia coli (strain K12) GN=en...,eno,45608.41069,35656.217111,1.626223e+09


## Calculate Cumulative abundance



     
Note that we do not calculate median copy number and muliply by molecular weight, we just fill genes by the median g/Protein.
The next version will include molecular weights for all proteins so we can do it by median copy number instead.


In [46]:
import numpy as np
b2u = pd.read_table('data/Ecoli/blatter-to-uniprot.tab', index_col = 'Blattner')
ecoli = df.join(b2u, on='gene_or_promoter').join(BW25113_fg, on='Uniprot')
medianAbundance = ecoli['g/Protein'].dropna().median()
ecoli.loc[ecoli['class']== 'promoter','g/Protein'] = 0
ecoli['g/Protein'].fillna(medianAbundance,inplace=True)
ecoli['cumulativeAbundance'] = np.cumsum(ecoli['g/Protein'])
                          
ecoli.to_csv('data/Ecoli/cumulative_abundance.tab', sep='\t', index=False)
ecoli

,gene_or_promoter,start,end,strand,class,genes_in_TU,start_if_select_as_start,cannot_as_start,Uniprot,Description,Gene,proteinMW,Copies/Cell_MG1655.Glucose,g/Protein,cumulativeAbundance
0,PM00249,148,189,1,promoter,"[b0001, b0002, b0003, b0004]",148,0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,0.000000e+00
1,b0001,190,255,1,gene,NaN,190,0,P0AD86,NaN,NaN,NaN,NaN,5.809959e+06,5.809959e+06
2,b0002,337,2799,1,gene,NaN,337,0,P00561,Bifunctional aspartokinase/homoserine dehydrog...,thrA,89045.79557,4375.280572,3.896003e+08,3.954103e+08
3,b0003,2801,3733,1,gene,NaN,2801,0,P00547,Homoserine kinase OS=Escherichia coli (strain ...,thrB,33583.89194,843.828359,2.833904e+07,4.237493e+08
4,b0004,3734,5020,1,gene,NaN,3734,0,P00934,Threonine synthase OS=Escherichia coli (strain...,thrC,47066.21410,13676.339450,6.436935e+08,1.067443e+09
5,b0005,5234,5530,1,gene,NaN,5234,0,P75616,NaN,NaN,NaN,NaN,5.809959e+06,1.073253e+09
6,b0006,5683,6459,-1,gene,NaN,5683,0,P0A8I3,UPF0246 protein yaaA OS=Escherichia coli (stra...,yaaA,29549.30240,612.865740,1.810976e+07,1.091363e+09
7,b0007,6529,7959,-1,gene,NaN,6529,0,P30143,NaN,NaN,NaN,NaN,5.809959e+06,1.097173e+09
8,PM0-9956,8191,8237,1,promoter,[b0008],8191,0,NaN,NaN,NaN,NaN,NaN,0.000000e+00,1.097173e+09
9,b0008,8238,9191,1,gene,NaN,8238,0,P0A870,Transaldolase B OS=Escherichia coli (strain K1...,talB,35179.30906,9247.453775,3.253190e+08,1.422492e+09


In [113]:
import pandas
abundance_f = 'data/Ecoli/{}_cumulative_mass.tab'.format(bw_glucose[0])
cum_abundance = pd.read_table(abundance_f, usecols=['gene_or_promoter', 'cumulativeMass']).set_index('gene_or_promoter')
cum_abundance

,cumulativeMass
gene_or_promoter,
PM00249,0.000000
b0001,0.017912
b0002,0.035824
b0003,0.068954
b0004,0.086866
b0005,0.104778
b0006,0.122690
b0007,0.140602
PM0-9956,0.140602


In [115]:
from IPython.display import display, Image, HTML
solution = pd.read_csv('out/local_result_essential.csv',index_col=0)
solution['end'] = solution['end'].str.lstrip('u_G_')
solution['start'] = solution['start'].str.lstrip('u_G_')
display( solution)

,end,start,status
0,b3340,b3322,Optimal
1,b0971,b0955,Optimal
2,b1636,b4493,Optimal
3,PM00633,PM0-10091,Optimal
4,b3559,b3465,Optimal
5,PM0-5862,PM00001,Optimal
6,b2779,b2765,Optimal
7,b0452,b0433,Optimal
8,PM324,b1289,Optimal
9,b2231,PM00652,Optimal


In [122]:
from IPython.display import Markdown
genome_view = []
mass_reclaimed = []
for i in solution.index:
    start, end = solution.loc[i, 'start'], solution.loc[i,'end']
    gv = ecoli.set_index('gene_or_promoter').loc[start:end].copy()
    mass_reclaimed.append(gv.loc[end,'cumulativeMass'] - gv.loc[start, 'cumulativeMass'])
    gv[r'mass reclaimed $(fg/cell)$'] = mass_reclaimed[i]
    gv['SolutionOrder'] = len(solution.index) - i
    genome_view.append( gv )
genome_view = pd.concat(genome_view)
genome_view.index.name = '$gene_or_promoter'
genome_view.to_csv('out/genome_view.tab',sep='\t',columns=['SolutionOrder', r'mass reclaimed $(fg/cell)$', bw_glucose[0], 'cumulativeMass'])
print('\n'.join([str(m) for m in mass_reclaimed]))
display(Markdown(r'Total mass reclaimed = %0.2g $fg/cell$' % sum(mass_reclaimed)))
HTML(genome_view.to_html())


24.412940987972263
7.079688593152859
4.832740699063095
4.193665944963769
3.6262855495866404
3.542662212617927
3.217499407842581
3.0853326347054058
3.0844945738922007
3.0549054483354894
2.90921138614776


Total mass reclaimed = 63 $fg/cell$

,start,end,strand,class,genes_in_TU,start_if_select_as_start,cannot_as_start,Uniprot,Description,Gene,Cellular protein location (according to www.uniprot.org),A14.07036,A14.07037,A14.07038,cumulativeMass,mass reclaimed $(fg/cell)$,SolutionOrder
$gene_or_promoter,,,,,,,,,,,,,,,,,
b3322,3453508,3453927,-1,gene,NaN,3453508,0,P03825,NaN,NaN,NaN,1.791195e-02,NaN,NaN,166.908478,24.412941,11
b3323,3453929,3455398,-1,gene,NaN,3453929,0,P45756,NaN,NaN,NaN,1.791195e-02,NaN,NaN,166.926390,24.412941,11
PM-8819,3455399,3455634,-1,promoter,"[b3322, b3323]",3455399,0,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,166.926390,24.412941,11
PM0-9514,3455482,3455577,1,promoter,"[b3335, b3334, b3333, b3332, b3331, b3330, b33...",3455634,1,NaN,NaN,NaN,NaN,0.000000e+00,NaN,NaN,166.926390,24.412941,11
b3324,3455578,3456393,1,gene,NaN,3455634,0,P45757,NaN,NaN,NaN,1.791195e-02,NaN,NaN,166.944302,24.412941,11
b3325,3456377,3458329,1,gene,NaN,3456393,0,P45758,NaN,NaN,NaN,1.791195e-02,NaN,NaN,166.962214,24.412941,11
b3326,3458339,3459820,1,gene,NaN,3458339,0,P45759,NaN,NaN,NaN,1.791195e-02,NaN,NaN,166.980126,24.412941,11
b3327,3459817,3461013,1,gene,NaN,3459820,0,P41441,NaN,NaN,NaN,1.791195e-02,NaN,NaN,166.998038,24.412941,11
b3328,3461023,3461460,1,gene,NaN,3461023,0,P41442,NaN,NaN,NaN,1.791195e-02,NaN,NaN,167.015950,24.412941,11
